In [1]:
import sys
import os  

RECORDS_PATH = '../input/tfrecords'

if os.path.exists('/kaggle'):
  !pip install tfrecord_lite
  sys.path.append('/kaggle/input/gezi-melt/utils')
  sys.path.append('/kaggle/input/official')
  from kaggle_datasets import KaggleDatasets
  try:
    GCS_PATH = KaggleDatasets().get_gcs_path('jigsaw-multilingual-toxic-comment-classification')
    RECORDS_PATH = KaggleDatasets().get_gcs_path('toxic-multi-tfrecords') + '/tfrecords'
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
  except Exception:
    RECORDS_PATH = '../input/toxic-multi-tfrecords/tfrecords'
    pass
!ls ../input
RECORDS_PATH

bert-multi					  tf-xlm-roberta-base
jigsaw-multilingual-toxic-comment-classification  tf-xlm-roberta-large
tfrecords


'../input/tfrecords'

In [2]:
import official
import gezi
import melt
import lele
import husky
import pandas as pd
import numpy as np
import tensorflow as tf

from absl import app, flags
FLAGS = flags.FLAGS
flags.DEFINE_string('model', None, '')
flags.DEFINE_bool('multi_head', False, '')
flags.DEFINE_string('pretrained', '../input/tf-xlm-roberta-large', '')
flags.DEFINE_integer('max_len', 192, 'xlm 192 bert 128')
flags.DEFINE_bool('freeze_pretrained', False, '')

tf.__version__

'2.2.0-dev20200411'

In [3]:
# flags
argv=['train.py']
FLAGS(argv)
mark='xlm'
FLAGS.train_input=f'{RECORDS_PATH}/{mark}/jigsaw-toxic-comment-train'
FLAGS.valid_input=f'{RECORDS_PATH}/{mark}/validation'
FLAGS.test_input=f'{RECORDS_PATH}/{mark}/test'
FLAGS.valid_interval_steps=100 
FLAGS.verbose=1 
FLAGS.num_epochs=1
FLAGS.keras=1 
FLAGS.buffer_size=2048
FLAGS.learning_rate=3e-5 
FLAGS.min_learning_rate=0.
FLAGS.opt_epsilon=1e-8 
FLAGS.optimizer='bert-adamw'
# FLAGS.optimizer='adam'
FLAGS.metrics=[] 
FLAGS.test_names=['id', 'toxic']
FLAGS.valid_interval_epochs=0.1
FLAGS.test_interval_epochs=1.
FLAGS.num_gpus=1
FLAGS.cache=0
FLAGS.model_dir='../working/exps/v1/base'
FLAGS.multi_head=0
FLAGS.batch_parse=1
FLAGS.save_model=0
# FLAGS.pretrained = '../input/tf-xlm-roberta-large/'
FLAGS.pretrained = '../input/tf-xlm-roberta-base/'
FLAGS.batch_size=16 if 'large' in FLAGS.pretrained else 32
FLAGS.debug=0

toxic_types = ['severe_toxic', 'obscene', 'identity_hate', 'threat', 'insult']

In [4]:
# evaluate
import pandas as pd
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.utils import shuffle

import gezi
logging = gezi.logging

langs = ['es', 'it', 'tr']

def evaluate(y_true, y_pred, x):
  try:
    y_true = y_true[:,0]
    y_pred = y_pred[:,0]
  except Exception:
    pass
  if y_pred.max() > 1. or y_pred.min() < 0:
    y_pred = gezi.sigmoid(y_pred)
  result = {}
  loss = log_loss(y_true, y_pred)
  result['loss'] = loss
  
  auc = roc_auc_score(y_true, y_pred)
  result['auc/all'] = auc
    
  if 'lang' in x:
    x['y_true'] = y_true
    x['pred'] = y_pred
    x['lang'] = gezi.decode(x['lang'])

    df = pd.DataFrame(x) 
    df = shuffle(df)
    logging.info('\n', df)

    for lang in langs:
      df_ = df[df.lang==lang]
      auc = roc_auc_score(df_.y_true, df_.pred)
      result[f'auc/{lang}'] = auc

  return result

In [5]:
# dataset
import tensorflow as tf
from tensorflow.keras import backend as K
import melt

class Dataset(melt.Dataset):
  def __init__(self, subset='valid', **kwargs):
    super(Dataset, self).__init__(subset, **kwargs)

  def parse(self, example):
    MAX_LEN = FLAGS.max_len
    features_dict = {
      'input_word_ids': tf.io.FixedLenFeature([MAX_LEN], tf.int64),
      'toxic': tf.io.FixedLenFeature([], tf.float32),
      'id': tf.io.FixedLenFeature([], tf.string),
      # 'comment_text': tf.io.FixedLenFeature([], tf.string), # TODO
    }

    def _adds(names, dtype=None, length=None):
      dtype_ = dtype
      for name in names:
        if name in self.example:
          dtype = dtype_ or self.example[name][0].dtype 
          if length:
            features_dict[name] = tf.io.FixedLenFeature([length], dtype)
          else:
            features_dict[name] = tf.io.FixedLenFeature([], dtype)

    _adds(['lang'], tf.string)

    _adds(['input_mask', 'all_segment_id'], tf.int64, MAX_LEN)
    
    _adds(toxic_types)

    features = self.parse_(serialized=example, features=features_dict)

    def _casts(names, dtype=tf.int32):
      for name in names:
        if name in features:
          features[name] = tf.cast(features[name], dtype)

    _casts(['input_word_ids', 'input_mask', 'all_segment_id'])
    
    x = features
    y = features['toxic']
#     y = tf.cast(features['toxic'] > 0.5, tf.float32)
    keys = ['toxic', *toxic_types]
    for key in keys:
      if key not in features:
        features[key] = tf.zeros_like(features['toxic'])
        
    _casts(toxic_types, tf.float32)
        
    melt.append_dim(features, keys)

    if FLAGS.multi_head:
      y = tf.concat([features[key] for key in keys], 1)

    return x, y
 

In [6]:
# loss
import tensorflow as tf

def calc_loss(y_true, y_pred):
  pass

def focal_loss(gamma=1.5, alpha=.2):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) - K.mean((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

def get_loss_fn():
#   return tf.compat.v1.losses.sigmoid_cross_entropy
  return tf.keras.losses.BinaryCrossentropy()
#   return focal_loss()

In [7]:
# model
import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Input

import melt
import gezi 
logging = gezi.logging

# class Model(keras.Model):
#   def __init__(self):
#     super(Model, self).__init__() 

#     self.bert_layer = bert_layer
#     dims = [32]
#     self.mlp = melt.layers.MLP(dims)
#     odim = len(toxic_types) + 1 if FLAGS.multi_head else 1
#     self.dense = keras.layers.Dense(odim, activation='sigmoid')

#   def call(self, input):
#     input_word_ids = input['input_word_ids']
#     input_mask = input['input_mask']
#     segment_ids = input['all_segment_id']
  
#     x, _ = self.bert_layer([input_word_ids, input_mask, segment_ids])
#     x = self.mlp(x)
#     x = self.dense(x)
#     return x

In [8]:
import transformers
from transformers import TFAutoModel

def xlm_model():
  pretrained = FLAGS.pretrained or XLM_PATH
  with gezi.Timer(f'load xlm_model from {pretrained}', True, logging.info):
    transformer = TFAutoModel.from_pretrained(pretrained)
  if FLAGS.freeze_pretrained:
    transformer.trainable = False
  input_word_ids = Input(shape=(FLAGS.max_len,), dtype=tf.int32, name="input_word_ids")
  sequence_output = transformer(input_word_ids)[0]
  cls_token = sequence_output[:, 0, :]
  odim = len(toxic_types) + 1 if FLAGS.multi_head else 1
  out = keras.layers.Dense(odim, activation='sigmoid')(cls_token)

  model = keras.Model(inputs=input_word_ids, outputs=out)

  return model

I0417 21:53:16.606246 139709059508032 file_utils.py:41] PyTorch version 1.4.0 available.
I0417 21:53:16.606915 139709059508032 file_utils.py:57] TensorFlow version 2.2.0-dev20200411 available.


In [9]:
Model = xlm_model

In [10]:
# train

import os
import melt

fit = melt.fit
melt.inited = False
melt.init()
loss_fn = get_loss_fn()

2020-04-17 21:53:20 0:00:00 fcntl.floc with lock_file /root/.melt.lock (If hang here means other programs calling melt.init have not finished yet)
2020-04-17 21:53:20 0:00:00 Tf dataset and Tf model train in Eager mode, keras 1, distributed:False
2020-04-17 21:53:20 0:00:00 log_level: 20 (try --debug to show more or --log_level=(> 20) to show less(no INFO), try --verbose to show train/valid loss intervaly)
2020-04-17 21:53:20 0:00:00 batch_size: 32 eval_batch_size: 32 batch_size_per_gpu: 32 num_gpus: 1 gpu: [0] CUDA_VISIABLE_DEVICES=[] work_mode: train distributed: False horovod: False
2020-04-17 21:53:21 0:00:01 model: [base] model_dir: [../working/exps/v1/base]


In [11]:
%%time 
strategy = melt.distributed.get_strategy()
with strategy.scope():
  model = Model()
try:
  model.summary()
except Exception:
  pass

2020-04-17 21:53:21 0:00:01 load xlm_model from ../input/tf-xlm-roberta-base/ start
I0417 21:53:21.834610 139709059508032 configuration_utils.py:281] loading configuration file ../input/tf-xlm-roberta-base/config.json
I0417 21:53:21.835743 139709059508032 configuration_utils.py:319] Model config XLMRobertaConfig {
  "_num_labels": 2,
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": 0,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": 2,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-05,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_em

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 192)]             0         
_________________________________________________________________
tf_roberta_model (TFRobertaM ((None, 192, 768), (None, 278043648 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 768)]             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 769       
Total params: 278,044,417
Trainable params: 278,044,417
Non-trainable params: 0
_________________________________________________________________
CPU times: user 9.17 s, sys: 3.03 s, total: 12.2 s
Wall time: 12.2 s


In [12]:
model.weights[-2][:10]

<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[ 0.0444],
       [ 0.0104],
       [ 0.0151],
       [ 0.0141],
       [-0.0071],
       [ 0.0694],
       [-0.0628],
       [ 0.0104],
       [-0.0612],
       [ 0.042 ]], dtype=float32)>

In [ ]:
# # model.load_weights('../input/toxic-multi/xlm.toxic.h5')
# checkpoint = tf.train.Checkpoint(model=model)
# x = checkpoint.restore(tf.train.latest_checkpoint('../working/exps/v1/xlm-unint2/'))

In [13]:
model.weights[-1]

<tf.Variable 'dense/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>

In [14]:
model.weights[0]

<tf.Variable 'tf_roberta_model/roberta/encoder/layer_._0/attention/self/query/kernel:0' shape=(768, 768) dtype=float32, numpy=
array([[-0.2859,  0.0218, -0.063 , ..., -0.1261,  0.0493, -0.0964],
       [-0.0558,  0.1133, -0.0298, ...,  0.016 ,  0.0894,  0.0027],
       [-0.0369,  0.1207,  0.2239, ...,  0.2328, -0.0745,  0.0836],
       ...,
       [-0.172 ,  0.1884, -0.0881, ...,  0.179 , -0.0625, -0.0113],
       [-0.0143, -0.0844,  0.0189, ..., -0.2363,  0.1165,  0.1169],
       [ 0.018 , -0.184 , -0.0734, ...,  0.1016,  0.1857,  0.0331]],
      dtype=float32)>

In [15]:
# try:
# model.save('/home/gezi/tmp/model', save_format='tf')
# except Exception:
#   model.save_weights('/home/gezi/tmp/model.h5')

In [16]:
# model.save_weights('/home/gezi/tmp/model.bin')

In [17]:
def run(model=model):
  fit(model,  
      loss_fn,
      Dataset,
      eval_fn=evaluate,
      eval_keys=['lang'],
      )   

In [18]:
# FLAGS.train_input=f'{RECORDS_GCS_PATH}/xlm-sample1/jigsaw-unintended-bias-train'
# FLAGS.learning_rate=3e-5
# FLAGS.opt_epsilon=1e-8
# FLAGS.num_epochs=1
# run()

In [19]:
# # 0.91485
FLAGS.train_input=f'{RECORDS_PATH}/{mark}/jigsaw-toxic-comment-train'
FLAGS.learning_rate=2e-5
FLAGS.opt_epsilon=1e-8
FLAGS.num_epochs=1  
FLAGS.num_train=10000
FLAGS.vie=0.5
FLAGS.tie=0
run()
FLAGS.num_train=None
# model.save('./xlm.toxic.3e5.h5')

2020-04-17 21:53:56 0:00:36 -------Round: 0 mode: None train_input:[jigsaw-toxic-comment-train] valid_input:[validation] train_dirs:[1] valid_dir: ../input/tfrecords/xlm/validation
2020-04-17 21:53:56 0:00:36 --start_hour=jigsaw-toxic-comment-train --end_hour=validation root: ../input/tfrecords/xlm
2020-04-17 21:53:57 0:00:37 num_train_examples: 10000 
2020-04-17 21:53:57 0:00:37 num_valid_examples: 8000  
2020-04-17 21:53:57 0:00:37 num_test_examples: 63812  
I0417 21:53:57.915369 139709059508032 optimization.py:111] using Adamw optimizer


  1/313 [..............................] - ETA: 0s - loss: 0.4864

2020-04-17 21:54:17 0:00:57 Model: "tf_roberta_model"
2020-04-17 21:54:17 0:00:57 _________________________________________________________________
2020-04-17 21:54:17 0:00:57 Layer (type)                 Output Shape              Param #   
2020-04-17 21:54:17 0:00:57 =================================================================
2020-04-17 21:54:17 0:00:57 roberta (TFRobertaMainLayer) ((None, 192, 768), (None, 278043648 
2020-04-17 21:54:17 0:00:57 =================================================================
2020-04-17 21:54:17 0:00:57 Total params: 278,043,648
2020-04-17 21:54:17 0:00:57 Trainable params: 278,043,648
2020-04-17 21:54:17 0:00:57 Non-trainable params: 0
2020-04-17 21:54:17 0:00:57 _________________________________________________________________
2020-04-17 21:54:17 0:00:57 Model: "model"
2020-04-17 21:54:17 0:00:57 _________________________________________________________________
2020-04-17 21:54:17 0:00:57 Layer (type)                 Output Shape            

155/313 [=============>................] - ETA: 1:55 - loss: 0.3461

eval:   0%|          | 0/250 [00:00<?, ?it/s]

...

eval:   0%|          | 0/250 [00:00<?, ?it/s]

 steps/s: 0.9 insts/s: 28.6 1epoch: 0.10h


312/313 [============================>.] - ETA: 1s - loss: 0.3115

2020-04-17 22:00:03 0:06:43 
      lang  y_true      pred
5581   it     0.0  0.018647
1848   tr     0.0  0.099293
1312   es     0.0  0.031786
5540   it     1.0  0.246988
3318   tr     0.0  0.104278
...   ...     ...       ...
1203   es     0.0  0.023875
5514   es     1.0  0.252196
6841   es     0.0  0.024332
3321   es     0.0  0.083272
5850   tr     0.0  0.167504

[8000 rows x 3 columns]
valid_metrics: [('loss', '0.38832'), ('auc/all', '0.77288'), ('auc/es', '0.77228'), ('auc/it', '0.76163'), ('auc/tr', '0.84298')]  num examples: 8000 eval_step: 2 step: 312 epoch: 1.0


313/313 [==============================] - 352s 1s/step - loss: 0.3113 - val_loss: 0.3643


In [21]:
model.weights[-2][:10]

<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[ 0.0443],
       [ 0.0103],
       [ 0.0149],
       [ 0.0141],
       [-0.0075],
       [ 0.0697],
       [-0.0629],
       [ 0.0103],
       [-0.0614],
       [ 0.0422]], dtype=float32)>

In [22]:
%%time 
strategy = melt.distributed.get_strategy()
with strategy.scope():
  model = Model()
try:
  model.summary()
except Exception:
  pass

2020-04-17 22:15:28 0:22:07 load xlm_model from ../input/tf-xlm-roberta-base/ start
I0417 22:15:28.130969 139709059508032 configuration_utils.py:281] loading configuration file ../input/tf-xlm-roberta-base/config.json
I0417 22:15:28.132275 139709059508032 configuration_utils.py:319] Model config XLMRobertaConfig {
  "_num_labels": 2,
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": 0,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": 2,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-05,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_em

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 192)]             0         
_________________________________________________________________
tf_roberta_model_1 (TFRobert ((None, 192, 768), (None, 278043648 
_________________________________________________________________
tf_op_layer_strided_slice_1  [(None, 768)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 769       
Total params: 278,044,417
Trainable params: 278,044,417
Non-trainable params: 0
_________________________________________________________________
CPU times: user 4.53 s, sys: 1.78 s, total: 6.31 s
Wall time: 6.29 s


In [23]:
model.weights[-1]

<tf.Variable 'dense_1/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>

In [29]:
!ls -l ../working/exps/v1/base/model.h5

-rw-r--r-- 1 root root 1112446992 4月  17 22:00 ../working/exps/v1/base/model.h5


In [30]:
model.load_weights('../working/exps/v1/base/model.h5')

In [34]:
checkpoint = tf.train.Checkpoint(model=model)
# x = checkpoint.restore(tf.train.latest_checkpoint('../working/exps/v1/xlm-unint2/'))
checkpoint.save('../working/exps/v1/base/ckpt')

'../working/exps/v1/base/ckpt-1'

In [58]:
with strategy.scope():
  model = Model()
model.weights[-1]

2020-04-17 23:04:35 1:11:15 load xlm_model from ../input/tf-xlm-roberta-base/ start
I0417 23:04:35.348017 139709059508032 configuration_utils.py:281] loading configuration file ../input/tf-xlm-roberta-base/config.json
I0417 23:04:35.349449 139709059508032 configuration_utils.py:319] Model config XLMRobertaConfig {
  "_num_labels": 2,
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": 0,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": 2,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-05,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_em

<tf.Variable 'dense_5/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>

In [40]:
checkpoint = tf.train.Checkpoint(model=model)
checkpoint.restore(tf.train.latest_checkpoint('../working/exps/v1/base/'))

In [46]:
model.weights[-1]

<tf.Variable 'dense_3/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>

In [42]:
tf.train.list_variables(tf.train.latest_checkpoint('../working/exps/v1/base/'))

[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('model/layer_with_weights-0/roberta/embeddings/LayerNorm/beta/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('model/layer_with_weights-0/roberta/embeddings/LayerNorm/gamma/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('model/layer_with_weights-0/roberta/embeddings/position_embeddings/embeddings/.ATTRIBUTES/VARIABLE_VALUE',
  [514, 768]),
 ('model/layer_with_weights-0/roberta/embeddings/token_type_embeddings/embeddings/.ATTRIBUTES/VARIABLE_VALUE',
  [1, 768]),
 ('model/layer_with_weights-0/roberta/embeddings/weight/.ATTRIBUTES/VARIABLE_VALUE',
  [250002, 768]),
 ('model/layer_with_weights-0/roberta/encoder/layer/0/attention/dense_output/LayerNorm/beta/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('model/layer_with_weights-0/roberta/encoder/layer/0/attention/dense_output/LayerNorm/gamma/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('model/layer_with_weights-0/roberta/encoder/layer/0/attention/dense_output/dense/bias/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('model/laye

In [43]:
tf.train.list_variables(tf.train.latest_checkpoint('../working/exps/v1/xlm-toxic/ckpt/'))

[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('layer_with_weights-0/roberta/embeddings/LayerNorm/beta/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('layer_with_weights-0/roberta/embeddings/LayerNorm/beta/.OPTIMIZER_SLOT/optimizer/m/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('layer_with_weights-0/roberta/embeddings/LayerNorm/beta/.OPTIMIZER_SLOT/optimizer/v/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('layer_with_weights-0/roberta/embeddings/LayerNorm/gamma/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('layer_with_weights-0/roberta/embeddings/LayerNorm/gamma/.OPTIMIZER_SLOT/optimizer/m/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('layer_with_weights-0/roberta/embeddings/LayerNorm/gamma/.OPTIMIZER_SLOT/optimizer/v/.ATTRIBUTES/VARIABLE_VALUE',
  [768]),
 ('layer_with_weights-0/roberta/embeddings/position_embeddings/embeddings/.ATTRIBUTES/VARIABLE_VALUE',
  [514, 768]),
 ('layer_with_weights-0/roberta/embeddings/position_embeddings/embeddings/.OPTIMIZER_SLOT/optimizer/m/.ATTRIBUTES/VARIABLE_VALUE',
  [514, 768]),


In [48]:
model.load_weights(tf.train.latest_checkpoint('../working/exps/v1/base/'))
model.weights[-1]

AssertionError: Nothing except the root object matched a checkpointed value. Typically this means that the checkpoint does not match the Python program. The following objects have no matching checkpointed value: [<tf.Variable 'tf_roberta_model_3/roberta/encoder/layer_._11/output/LayerNorm/beta:0' shape=(768,) dtype=float32, numpy=
array([ 0.0213,  0.0468,  0.0323,  0.0319,  0.0368,  0.0142,  0.0249,
        0.0349,  0.0369,  0.0113,  0.0396,  0.0307, -0.0427,  0.0122,
        0.0166,  0.037 ,  0.018 ,  0.0252,  0.045 ,  0.029 ,  0.019 ,
        0.0193,  0.0391,  0.0395,  0.0298,  0.0196,  0.0296,  0.0285,
        0.0369,  0.0282,  0.0487,  0.0223,  0.0251,  0.0356,  0.0363,
        0.0213,  0.0204,  0.0181,  0.048 ,  0.0369,  0.048 ,  0.0334,
        0.0363,  0.0351,  0.024 ,  0.0116,  0.0258,  0.0154,  0.0302,
        0.0108,  0.0192,  0.0259,  0.0414,  0.0698,  0.0233,  0.0241,
        0.0427,  0.0312,  0.021 ,  0.0538,  0.008 ,  0.0271,  0.0269,
        0.0386,  0.0304,  0.0212,  0.

In [62]:
model.load_weights(tf.train.latest_checkpoint('../working/exps/v1/xlm-toxic/ckpt/'))
model.weights[-1]

<tf.Variable 'dense_5/bias:0' shape=(1,) dtype=float32, numpy=array([-0.0003], dtype=float32)>

In [64]:
[x.name for x in model.variables]

['tf_roberta_model_5/roberta/encoder/layer_._0/attention/self/query/kernel:0',
 'tf_roberta_model_5/roberta/encoder/layer_._0/attention/self/query/bias:0',
 'tf_roberta_model_5/roberta/encoder/layer_._0/attention/self/key/kernel:0',
 'tf_roberta_model_5/roberta/encoder/layer_._0/attention/self/key/bias:0',
 'tf_roberta_model_5/roberta/encoder/layer_._0/attention/self/value/kernel:0',
 'tf_roberta_model_5/roberta/encoder/layer_._0/attention/self/value/bias:0',
 'tf_roberta_model_5/roberta/encoder/layer_._0/attention/output/dense/kernel:0',
 'tf_roberta_model_5/roberta/encoder/layer_._0/attention/output/dense/bias:0',
 'tf_roberta_model_5/roberta/encoder/layer_._0/attention/output/LayerNorm/gamma:0',
 'tf_roberta_model_5/roberta/encoder/layer_._0/attention/output/LayerNorm/beta:0',
 'tf_roberta_model_5/roberta/encoder/layer_._0/intermediate/dense/kernel:0',
 'tf_roberta_model_5/roberta/encoder/layer_._0/intermediate/dense/bias:0',
 'tf_roberta_model_5/roberta/encoder/layer_._0/output/den

In [65]:
optimizer = gezi.get('optimizer')

In [69]:
[x.name for x in optimizer.variables()]

['iter:0',
 'AdamWeightDecay/tf_roberta_model/roberta/encoder/layer_._0/attention/self/query/kernel/m:0',
 'AdamWeightDecay/tf_roberta_model/roberta/encoder/layer_._0/attention/self/query/bias/m:0',
 'AdamWeightDecay/tf_roberta_model/roberta/encoder/layer_._0/attention/self/key/kernel/m:0',
 'AdamWeightDecay/tf_roberta_model/roberta/encoder/layer_._0/attention/self/key/bias/m:0',
 'AdamWeightDecay/tf_roberta_model/roberta/encoder/layer_._0/attention/self/value/kernel/m:0',
 'AdamWeightDecay/tf_roberta_model/roberta/encoder/layer_._0/attention/self/value/bias/m:0',
 'AdamWeightDecay/tf_roberta_model/roberta/encoder/layer_._0/attention/output/dense/kernel/m:0',
 'AdamWeightDecay/tf_roberta_model/roberta/encoder/layer_._0/attention/output/dense/bias/m:0',
 'AdamWeightDecay/tf_roberta_model/roberta/encoder/layer_._0/attention/output/LayerNorm/gamma/m:0',
 'AdamWeightDecay/tf_roberta_model/roberta/encoder/layer_._0/attention/output/LayerNorm/beta/m:0',
 'AdamWeightDecay/tf_roberta_model/rob

In [55]:
model.save_weights('../working/exps/v1/base/model')

In [61]:
model.load_weights('../working/exps/v1/base/model')
model.weights[-1]

<tf.Variable 'dense_5/bias:0' shape=(1,) dtype=float32, numpy=array([-0.0003], dtype=float32)>

In [51]:
checkpoint = tf.train.Checkpoint(model=model)
checkpoint.restore(tf.train.latest_checkpoint('../working/exps/v1/xlm-toxic/ckpt/'))
model.weights[-1]

<tf.Variable 'dense_4/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>

In [52]:
checkpoint = tf.train.Checkpoint()
checkpoint.restore(tf.train.latest_checkpoint('../working/exps/v1/xlm-toxic/ckpt/'))
model.weights[-1]

<tf.Variable 'dense_4/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>

In [ ]:
FLAGS.valid_input=f'{RECORDS_PATH}/{mark}/validation'
FLAGS.train_input=FLAGS.valid_input
FLAGS.learning_rate=1e-5
FLAGS.num_folds = 1
FLAGS.vie=0.2
FLAGS.ev_first=True
FLAGS.num_train=1
run()
FLAGS.num_folds = None
FLAGS.vie=0.1
FLAGS.valid_input = FLAGS.train_input

In [70]:
model.save_weights('../working/exps/v1/base/tmp/model_weight.ckpt')

In [71]:
model.save_weights('../working/exps/v1/base/tmp/model_weight.h5')

In [73]:
# model.save('../working/exps/v1/base/tmp/model.h5')
model.save('../working/exps/v1/base/tmp/model')

In [74]:
tf.keras.models.save_model(model, '../working/exps/v1/base/tmp/model2')

In [ ]:
model.weights[0]

In [ ]:
FLAGS.train_input=FLAGS.valid_input
FLAGS.learning_rate=1e-5
FLAGS.num_folds = 5
FLAGS.vie=1.
run()
FLAGS.num_folds = None
FLAGS.vie=0.1
FLAGS.valid_input = FLAGS.train_input

In [ ]:
# model.save_weights('./xlm.toxic-uint1.h5')

In [ ]:
# FLAGS.train_input=FLAGS.valid_input
# FLAGS.learning_rate=1e-5
# FLAGS.opt_epsilon=1e-8
# FLAGS.num_epochs=1
# run()

In [ ]:
# model.save_weights('./xlm.final.h5')

In [ ]:
# # with strategy.scope():
# #   model = Model()
# FLAGS.train_input=f'{RECORDS_GCS_PATH}/{mark}/jigsaw-toxic-comment-train,{RECORDS_GCS_PATH}/xlm-sample2/jigsaw-unintended-bias-train'
# # FLAGS.train_input=f'{RECORDS_GCS_PATH}/xlm/jigsaw-unintended-bias-train'
# FLAGS.learning_rate=3e-5
# FLAGS.opt_epsilon=1e-8
# FLAGS.num_epochs=1  
# FLAGS.valid_interval_epochs=0.1
# run()

In [ ]:
# FLAGS.train_input=FLAGS.valid_input
# FLAGS.learning_rate=3e-5
# FLAGS.opt_epsilon=1e-8
# FLAGS.num_epochs=1
# FLAGS.valid_interval_epochs=0.2
# FLAGS.optimizer='bert-adamw'
# run()

In [ ]:
d = pd.read_csv('../working/exps/v1/base/submission.csv')
d

In [ ]:
# batch_size = melt.batch_size()
# inputs = gezi.list_files(FLAGS.train_input)
# train_dataset_ = Dataset('train')
# train_dataset = train_dataset_.make_batch(batch_size, inputs, repeat=True, cache=True)
# valid_inputs = gezi.list_files(FLAGS.valid_input)
# valid_dataset_ = Dataset('valid')
# valid_dataset = valid_dataset_.make_batch(batch_size, valid_inputs, shuffle=False, repeat=False, cache=True)
# test_inputs = gezi.list_files(FLAGS.test_input)
# test_dataset_ = Dataset('test')
# test_dataset = test_dataset_.make_batch(batch_size, test_inputs, shuffle=False, repeat=False, cache=True)

In [ ]:
d2 = pd.read_csv('../input/toxic-multi-v1-base/submission.csv')
d2.toxic

In [ ]:
gezi.metrics.inverse_ratio(d.toxic.values, d2.toxic.values)

In [ ]:
d.to_csv('submission.csv', index=False)
d.head()

In [ ]:
# # training step function
# loss_fn = tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
# @tf.function
# def train_step(x, y):
#     with tf.GradientTape() as tape:
#         y_ = model(x)
#         loss = loss_fn(y, y_)
#         loss = tf.reduce_sum(loss * (1. / melt.batch_size()))
#     grads = tape.gradient(loss, model.trainable_variables)
#     optimizer.apply_gradients(zip(grads, model.trainable_variables))

# # distributed dataset
# train_dist_ds = strategy.experimental_distribute_dataset(train_dataset)

# # custom training loop
# for (x, y) in train_dist_ds:
#     strategy.experimental_run_v2(train_step, args=(x, y))